In [1]:
import numpy as np
import cv2
from skimage import exposure

def multi_otsu_thresholding(image, thresholds):
    """
    Perform multi-level Otsu thresholding using given thresholds.
    
    Args:
        image (np.ndarray): Grayscale image.
        thresholds (list): List of thresholds for segmentation.

    Returns:
        segmented_img (np.ndarray): Segmented image based on thresholds.
    """
    thresholds = np.sort(thresholds)
    num_thresholds = len(thresholds)
    thresholds = np.concatenate(([0], thresholds, [255]))  # Add boundaries

    # Compute histogram and probabilities
    hist, bins = np.histogram(image.flatten(), bins=256, range=(0, 256))
    total_pixels = image.size
    probabilities = hist / total_pixels
    
    # Compute cumulative sums
    cumulative_sum = np.cumsum(probabilities)
    cumulative_mean = np.cumsum(probabilities * np.arange(256))
    
    # Compute global mean
    global_mean = cumulative_mean[-1]
    
    # Compute between-class variance for each class
    class_variances = []
    for i in range(num_thresholds + 1):
        p1 = cumulative_sum[thresholds[i]]
        p2 = cumulative_sum[thresholds[i + 1]] - p1
        if p1 == 0 or p2 == 0:
            class_variances.append(0)
            continue
        
        mean1 = cumulative_mean[thresholds[i]] / p1
        mean2 = (cumulative_mean[thresholds[i + 1]] - cumulative_mean[thresholds[i]]) / p2
        
        variance = p1 * p2 * (mean1 - mean2) ** 2
        class_variances.append(variance)
    
    # Total variance is the sum of variances for all classes
    total_variance = np.sum(class_variances)
    
    return total_variance



In [2]:
import numpy as np

# PSO Parameters
SWARM_SIZE = 30
DIMENSIONS = 2  # Number of thresholds (2 thresholds for 3 classes)
MAX_ITERATIONS = 100
W = 0.5
C1 = 1.5
C2 = 1.5
RANGE = 255  # Maximum value for a threshold

def pso_algorithm(image):
    # Initialize particles
    swarm = np.random.uniform(0, RANGE, (SWARM_SIZE, DIMENSIONS))
    velocities = np.random.uniform(-RANGE, RANGE, (SWARM_SIZE, DIMENSIONS))
    personal_best_positions = np.copy(swarm)
    personal_best_values = np.array([multi_otsu_thresholding(image, pos) for pos in swarm])
    
    global_best_index = np.argmin(personal_best_values)
    global_best_position = personal_best_positions[global_best_index]
    global_best_value = personal_best_values[global_best_index]
    
    for t in range(MAX_ITERATIONS):
        # Update velocities and positions
        r1, r2 = np.random.rand(SWARM_SIZE, DIMENSIONS), np.random.rand(SWARM_SIZE, DIMENSIONS)
        velocities = (W * velocities +
                      C1 * r1 * (personal_best_positions - swarm) +
                      C2 * r2 * (global_best_position - swarm))
        swarm += velocities
        
        # Apply boundary conditions
        swarm = np.clip(swarm, 0, RANGE)
        
        # Evaluate fitness
        fitness_values = np.array([multi_otsu_thresholding(image, pos) for pos in swarm])
        
        # Update personal bests
        better_mask = fitness_values < personal_best_values
        personal_best_positions[better_mask] = swarm[better_mask]
        personal_best_values[better_mask] = fitness_values[better_mask]
        
        # Update global best
        current_best_index = np.argmin(personal_best_values)
        current_best_value = personal_best_values[current_best_index]
        if current_best_value < global_best_value:
            global_best_position = personal_best_positions[current_best_index]
            global_best_value = current_best_value
        
        # Print iteration details
        print(f"Iteration {t+1}")
        print("  Global Best Position and Value:")
        print(f"    Global Best Position: {global_best_position}, Global Best Value: {global_best_value}\n")
        
    return global_best_position, global_best_value



In [5]:
import cv2
import matplotlib.pyplot as plt

# Read the image in grayscale
image_path = './dataset/2.BMP'
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Run PSO to find the optimal thresholds
best_thresholds, _ = pso_algorithm(img)

# Segment the image using the best thresholds found
def segment_image(image, thresholds):
    thresholds = np.sort(thresholds).astype(np.uint8) 
    print(f"Thresholds: {thresholds}")
    num_thresholds = len(thresholds)
    thresholds = np.concatenate(([0], thresholds, [255]))  # Add boundaries

    segmented_img = np.zeros_like(image, dtype=np.uint8)
    for i in range(num_thresholds + 1):
        if i == 0:
            segmented_img[(image < thresholds[i + 1])] = 0
        elif i == num_thresholds:
            segmented_img[(image >= thresholds[i])] = 255
        else:
            segmented_img[(image >= thresholds[i]) & (image < thresholds[i + 1])] = (i + 1) * (255 // (num_thresholds + 1))
    
    return segmented_img

segmented_img = segment_image(img, best_thresholds)

# Display the results
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(segmented_img, cmap='gray')
plt.title('Segmented Image with PSO')
plt.axis('off')

plt.show()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# PSO Parameters
SWARM_SIZE = 30
DIMENSIONS = 2  # Number of thresholds (2 thresholds for 3 classes)
MAX_ITERATIONS = 100
W = 0.5
C1 = 1.5
C2 = 1.5
RANGE = 255  # Maximum value for a threshold

# Multi-level Otsu thresholding fitness function
def multi_otsu_thresholding(image, thresholds):
    thresholds = np.sort(thresholds)
    num_thresholds = len(thresholds)
    thresholds = np.concatenate(([0], thresholds, [255]))  # Add boundaries

    # Compute histogram and probabilities
    hist, _ = np.histogram(image.flatten(), bins=256, range=(0, 256))
    total_pixels = image.size
    probabilities = hist / total_pixels
    
    # Compute cumulative sums and means
    cumulative_sum = np.cumsum(probabilities)
    cumulative_mean = np.cumsum(probabilities * np.arange(256))

    # Compute between-class variance for each class
    class_variances = []
    for i in range(num_thresholds + 1):
        p1 = cumulative_sum[thresholds[i]]
        p2 = cumulative_sum[thresholds[i + 1]] - p1
        if p1 == 0 or p2 == 0:
            class_variances.append(0)
            continue
        
        mean1 = cumulative_mean[thresholds[i]] / p1 if p1 > 0 else 0
        mean2 = (cumulative_mean[thresholds[i + 1]] - cumulative_mean[thresholds[i]]) / p2 if p2 > 0 else 0

        variance = p1 * p2 * (mean1 - mean2) ** 2
        class_variances.append(variance)
    
    total_variance = np.sum(class_variances)
    return total_variance

# PSO algorithm to optimize the thresholds
def pso_algorithm(image):
    # Initialize particles and velocities
    swarm = np.random.uniform(0, RANGE, (SWARM_SIZE, DIMENSIONS))
    velocities = np.random.uniform(-RANGE, RANGE, (SWARM_SIZE, DIMENSIONS))
    personal_best_positions = np.copy(swarm)
    personal_best_values = np.array([multi_otsu_thresholding(image, pos) for pos in swarm])
    
    # Initialize global best
    global_best_index = np.argmin(personal_best_values)
    global_best_position = personal_best_positions[global_best_index]
    global_best_value = personal_best_values[global_best_index]
    
    for t in range(MAX_ITERATIONS):
        # Update velocities and positions
        r1, r2 = np.random.rand(SWARM_SIZE, DIMENSIONS), np.random.rand(SWARM_SIZE, DIMENSIONS)
        velocities = (W * velocities +
                      C1 * r1 * (personal_best_positions - swarm) +
                      C2 * r2 * (global_best_position - swarm))
        swarm += velocities
        
        # Apply boundary conditions
        swarm = np.clip(swarm, 0, RANGE)
        
        # Evaluate fitness for each particle
        fitness_values = np.array([multi_otsu_thresholding(image, pos) for pos in swarm])
        
        # Update personal bests
        better_mask = fitness_values < personal_best_values
        personal_best_positions[better_mask] = swarm[better_mask]
        personal_best_values[better_mask] = fitness_values[better_mask]
        
        # Update global best if better solution is found
        current_best_index = np.argmin(personal_best_values)
        current_best_value = personal_best_values[current_best_index]
        if current_best_value < global_best_value:
            global_best_position = personal_best_positions[current_best_index]
            global_best_value = current_best_value
        
        # Print iteration details
        print(f"Iteration {t+1}: Global Best Position: {global_best_position}, Global Best Value: {global_best_value}\n")
        
    return global_best_position, global_best_value

# Segment the image using the best thresholds found
def segment_image(image, thresholds):
    thresholds = np.sort(thresholds).astype(int)  # Ensure thresholds are integers
    num_thresholds = len(thresholds)
    thresholds = np.concatenate(([0], thresholds, [255]))  # Add boundaries

    segmented_img = np.zeros_like(image, dtype=np.uint8)
    for i in range(num_thresholds + 1):
        if i == 0:
            segmented_img[(image < thresholds[i + 1])] = 0
        elif i == num_thresholds:
            segmented_img[(image >= thresholds[i])] = 255
        else:
            segmented_img[(image >= thresholds[i]) & (image < thresholds[i + 1])] = (i + 1) * (255 // (num_thresholds + 1))
    
    return segmented_img

# Main function to run the PSO-based segmentation
def main():
    # Read the image in grayscale
    image_path = 'lena_color.tiff'
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Run PSO to find the optimal thresholds
    best_thresholds, _ = pso_algorithm(img)

    # Segment the image using the best thresholds found
    segmented_img = segment_image(img, best_thresholds)

    # Display the results
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(segmented_img, cmap='gray')
    plt.title('Segmented Image with PSO')
    plt.axis('off')

    plt.show()

# Run the main function
if __name__ == "__main__":
    main()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices